In [ ]:
from openai import OpenAI
import json
import minsearch

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
def create_index(documents):
    index = minsearch.Index(
        text_fields=['question', 'text', 'section'],
        keyword_fields=['course'],
    )
    index.fit(documents)
    return index

In [ ]:
index = create_index(documents)

In [3]:
def search(query, course='data-engineering-zoomcamp', num_results=5):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query='the course has already started, can I still enroll?',
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=num_results
    )
    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
    Use only the facts from the CONTEXT while answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [7]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
query = 'How do I run kafka?'

rag(query)